In [2]:
pip install arduino-iot-cloud dash pandas numpy plotly

Note: you may need to restart the kernel to use updated packages.


In [14]:
# Live Dashboard for Sensor Data (no CSV saving)

from jupyter_dash import JupyterDash
from dash import html, dcc, Input, Output
import plotly.express as px
import pandas as pd
from collections import deque
import threading, time
from arduino_iot_cloud import ArduinoCloudClient

# ==== Arduino IoT Cloud settings ====
DEVICE_ID  = "4eba5b6b-d6f4-4d8b-a168-661690be2aa9"
SECRET_KEY = "Fb3VV?Nb79rYwGkb@kHHEZkV#"
VAR_X, VAR_Y, VAR_Z = "py_x", "py_y", "py_z"

# ==== Data & buffers ====
N_BATCH = 10
ingest_A: deque = deque()
plot_B = []

def on_sensor_sample(x, y, z, ts_ms=None):
    if ts_ms is None:
        ts_ms = int(time.time() * 1000)
    sample = {"ts_ms": int(ts_ms), "x": float(x), "y": float(y), "z": float(z)}
    ingest_A.append(sample)

def move_N_from_A_to_B(N=N_BATCH):
    """Move exactly N samples to plot_B if available."""
    global plot_B
    if len(ingest_A) >= N:
        plot_B = [ingest_A.popleft() for _ in range(N)]
        return True
    return False

def df_from_plot_B():
    if not plot_B:
        return pd.DataFrame(columns=["ts_ms", "t_seconds", "x", "y", "z"])
    df = pd.DataFrame(plot_B)
    t0 = df["ts_ms"].iloc[0]
    df["t_seconds"] = (df["ts_ms"] - t0) / 1000.0
    return df

# ==== Cloud polling thread (SYNC MODE) ====
def _to_float(v):
    try:
        return float(v)
    except (TypeError, ValueError):
        return None

def poll_cloud_client(stop_event: threading.Event, poll_interval=0.10):
    client = ArduinoCloudClient(
        device_id=DEVICE_ID,
        username=DEVICE_ID,
        password=SECRET_KEY,
        sync_mode=True,        # avoids async/loop conflicts in notebooks
    )
    client.register(VAR_X, value=None)
    client.register(VAR_Y, value=None)
    client.register(VAR_Z, value=None)
    client.start()
    try:
        while not stop_event.is_set():
            client.update()
            vx = _to_float(client.get(VAR_X))
            vy = _to_float(client.get(VAR_Y))
            vz = _to_float(client.get(VAR_Z))
            if vx is not None and vy is not None and vz is not None:
                on_sensor_sample(vx, vy, vz)
            time.sleep(poll_interval)
    finally:
        pass  # client.stop() if supported

stop_event = threading.Event()
threading.Thread(target=poll_cloud_client, args=(stop_event,), daemon=True).start()

# ==== Dash app ====
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H3("Data’s Live Dashboard"),
    html.Div(className="controls", children=[
        html.Label("Axes to plot:"),
        dcc.Checklist(
            id="axis-select",
            options=[{"label": "X", "value": "x"},
                     {"label": "Y", "value": "y"},
                     {"label": "Z", "value": "z"}],
            value=["x", "y", "z"],
            inline=True
        ),
    ]),
    html.Hr(),
    html.Div("Sensor Graph"),
    dcc.Graph(id="sensor-graph"),
    html.Hr(),
    html.Div(id="status-label"),
    dcc.Interval(id="tick", interval=1000, n_intervals=0),
])

@app.callback(
    Output("sensor-graph", "figure"),
    Output("status-label", "children"),
    Input("tick", "n_intervals"),
    Input("axis-select", "value"),
)
def refresh(_n, axes_selected):
    moved = move_N_from_A_to_B(N_BATCH)
    df = df_from_plot_B()
    status = f"Ingest A: {len(ingest_A)} samples | Plot B: {len(df)} samples"
    if moved:
        status += " | New batch ready"

    if df.empty or not axes_selected:
        fig = px.line(title="Waiting for data…")
        return fig, status

    long_df = df.melt(id_vars=["t_seconds"], value_vars=["x", "y", "z"],
                      var_name="axis", value_name="value")
    long_df = long_df[long_df["axis"].isin(axes_selected)]
    axis_map = {"x": "X", "y": "Y", "z": "Z"}
    long_df["axis"] = long_df["axis"].map(axis_map).fillna(long_df["axis"])

    fig = px.line(long_df, x="t_seconds", y="value", color="axis",
                  title="Sensor Data (last batch)")
    if len(long_df) <= 2000:
        fig.update_traces(mode="lines+markers")
    fig.update_xaxes(title="Seconds")
    fig.update_yaxes(title="Value")
    return fig, status

# Run inline in Jupyter/Colab
app.run_server(mode="inline", debug=False)

# When finished (optional): stop_event.set()

/Users/arsalansharifizad/anaconda3/lib/python3.13/site-packages/dash/dash.py:634: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



AttributeError: 'super' object has no attribute 'run_server'